In [1]:
import sys
from pathlib import Path
sys.path.append(Path.cwd().joinpath('src'))

In [2]:
import numpy as np
from rips import *
from rips.utils import FeynmanKac, Particle

# Setting model

In [48]:
# Define linear limit state function
class LinearLSF(FeynmanKac, StandardGaussian):
    def __init__(self, r=1, n=10, **kwargs):
        self.r = r
        self.n = n
        super().__init__(**kwargs)
    
    def score_function(self, particle: Particle) -> float:
        particle.response = particle.path.sum() / np.sqrt(self.n)
        return particle.response / self.r
    
    @property
    def num_variables(self) -> int:
        return self.n

model = LinearLSF(r=2, n=1000)

In [52]:
# Sample a particle
particle = model.sample_particle()

In [53]:
# This is the score function evaluated on a random vector
particle.score  # Greater than one means failure

0.10019034144427179

# Probabilistic inference

In [62]:
# Number of samples
model.num_of_particles = 10000
# Markov Chain Monte Carlo methods
kernels = [
    PCN(),
    MMH()
]
# Sample-based Monte Carlo integration
sample = ComboUQ(model, kernels)
sample.summary_results()


Adaptive levels...
[-inf, 0.42164209064034186]
[-inf, 0.42164209064034186, 0.8781845826247363]
[-inf, 0.42164209064034186, 0.8781845826247363, 1.2041776503302515]
[-inf, 0.42164209064034186, 0.8781845826247363, 1]
Fixed levels...
done


{'cpu_time_smc': 3.2913532079999186,
 'ng_smc': 170004,
 'cpu_time_gs': 3.0766673330001595,
 'p_bar': 0.022448000000000003,
 'ng_gs': 50130,
 'k': 3,
 'var': 0.0070449517911791186,
 'std': 0.08393421108927586}

# Compare against exact solution

In [63]:
from scipy.stats import norm
p_exact = norm.cdf(-2)
sample.p_smc

0.022776

In [67]:
type(sample.__dict__['p_smc']) in [numpy.float64]

NameError: name 'numpy' is not defined